In [10]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam

from nucleus.models import NeuralNetwork
from nucleus.data_loader import ElectronSpectrum
from nucleus.data_augment import generate_training_data

In [2]:
data_dir = './dataset/PDD.csv'
df = pd.read_csv(data_dir, index_col=False)


PDD = df.iloc[:, 0].to_numpy()
matrix_R = df.iloc[:, 1:].to_numpy()

In [4]:
num_samples = 10000
parameters = {
        'I0' : np.random.uniform(0.000001, 0.0003),
        'A' : np.random.uniform(0.01, 0.1),
        'Ep' : np.random.uniform(1, 11),
        'w' : np.random.uniform(0.001, 0.1)
}
E_values = np.arange(1, 11.05, 0.05)  # 201 giá trị năng lượng
noise_stddev=0.03

X_train, Y_train = generate_training_data(
    num_samples=num_samples,
    matrix_R=matrix_R,
    E_values=E_values,
    parameters=parameters,
    noise_stddev=noise_stddev,
)


In [6]:
# Chuẩn hóa dữ liệu đầu vào X_train
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)

In [11]:
X_train.shape

(10000, 48)

In [12]:
model = NeuralNetwork(in_features=X_train.shape[1], 
                      out_features=4, 
                      mid_features=128, 
                      n_layers=5, 
                      device='cuda' if torch.cuda.is_available() else 'cpu')    

In [19]:
sample = torch.tensor(X_train[0], device='cuda', dtype=torch.float32)
model(sample)

tensor([0.0000, 0.0000, 0.0327, 0.0000], device='cuda:0',
       grad_fn=<ReluBackward0>)

In [ ]:
# Khởi tạo optimizer
learning_rate = 0.00001
optimizer = Adam(model.parameters(), lr=learning_rate)

# Khởi tạo hàm loss
loss_fn = nn.MSELoss()

# Tạo DataLoader
train_dataset = TensorDataset(torch.tensor(X_train, device='cuda', 
                                           dtype=torch.float32), torch.tensor(Y_train))
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# Hàm huấn luyện
def train(model, train_loader, optimizer, loss_fn, epochs=500):
    """
    Hàm huấn luyện mô hình.

    Args:
        model: Mô hình mạng neural.
        train_loader: DataLoader cho tập dữ liệu huấn luyện.
        optimizer: Optimizer được sử dụng để cập nhật trọng số.
        loss_fn: Hàm loss được sử dụng để tính toán lỗi.
        epochs: Số lượng epochs để huấn luyện.
    """
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            # Đặt mô hình ở chế độ huấn luyện
            model.train()

            # Tính toán dự đoán
            output = model(data)
            # Tính toán loss
            loss = loss_fn(output, target)

            # Cập nhật trọng số
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # In ra thông tin huấn luyện
            if batch_idx % 100 == 0:
                print(f'Epoch {epoch+1}/{epochs}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}')

# Huấn luyện mô hình
train(model, train_loader, optimizer, loss_fn)

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float